In [1]:
pwd

'/Users/ozkansafak/code/fraktur/notebooks'

In [2]:
%%time
%load_ext autoreload
%autoreload 2

# import standard Python modules
import time 
import numpy as np
import os, sys
import re
import glob
import openai

# # Get the root path of the project
sys.path.append(os.path.abspath(".."))

# Import necessary modules from your src directory
from src.utils import timeit, encode_image, plt
from src.processing import compute_log_spectrum_1d, extract_image_bbox, save_images
from src.api_requests import construct_payload, send_gpt_request
from src.document_generation import save_document
from src.api_requests import single_page

from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% !important; }</style>"))

print('sys.executable:', sys.executable)
print('sys.version:', sys.version, '\n')

<timed exec>:22: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display


sys.executable: /Users/ozkansafak/code/fraktur/.venv/bin/python3
sys.version: 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ] 

CPU times: user 340 ms, sys: 58.2 ms, total: 399 ms
Wall time: 513 ms


---
## Fraktur Translator

In [3]:
# Set the OpenAI API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

# Set the headers for the API request
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}

# Select the value of model_name in ["gpt-4o-2024-08-06", "gpt-4o-mini-2024-07-18"]
model_name = "gpt-4o-2024-08-06"
plotter = False

# Folder containing the Fraktur PDF files
image_path = "../data/Der Weltkrieg v8 Chap 4F (in Fraktur)/*pdf"

# Get the list of PDF file paths
fnames = sorted(glob.glob(image_path))
german_texts = {}
english_texts = {}
total_time = 0 

In [8]:
for i, fname in enumerate(fnames[13:]):
    start = time.time()

    pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
    print(f"\n{'>'*10} page:{pageno} (i:{i}/{len(fnames)-1}) ", end="")
    
    raw_text, raw_german_text, german_text, english_text = single_page(fname, model_name, headers, plotter, pageno)
    
    german_texts[pageno] = german_text
    english_texts[pageno] = english_text
    delta = time.time() - start 
    total_time += delta
    print(f"[time:{delta:.2f}sec] [TotalRuntime:{int(total_time//60)} mins {(total_time - int(total_time//60)*60):.2f} sec] {'>'*10}\n")
    print(raw_text)


>>>>>>>>>> page:469 (i:0/83) [time:35.56sec] [TotalRuntime:7 mins 53.24 sec] >>>>>>>>>>

<raw_german>
Flottenvorstoß in den Rigaer Busen. 469
Maßnahmen der Russen1).
Bald nachdem die Angriffe des Generals von Below gegen Mitte Juni nachgelassen hatten, begannen die Russen das Gebiet westlich von Riga und diese wichtige Stadt selbst zu räumen2). Sie führten einen großen Teil der zahlreich jüdischen Bevölkerung weg und schafften alles fort, was für die Kriegführung wichtig war, vor allem die Maschinen der Rigaer Fabriken3). Zugleich haben sie sich durch die Bedürfnisse der Hauptkampffront gezwungen, ihre militärischen Kräfte nördlich des Njemen wieder zu schwächen. Allein drei Infanterie-Divisionen3) wurden bis Ende Juni von dort in die Njemen-Festungen Kowno, Olita und Grodno verlegt, 2½ weitere in der ersten Julihälfte an andere Fronten ab- befördert4). So verfügte General Plehwe, als am 14. Juli der neue deutsche Angriff über die Düna in der Richtung auf Mitau vorbrach5), im ganzen no

In [15]:
(german_texts['457'])

'\n<header>Aufgabe und Gliederung der Njemen-Armee.</header>\n<body>\nDie N j e m e n - A r m e e unter General der Infanterie O t t o\nv o n B e l o w deckte zu dieser Zeit mit rund sieben Infanterie-Divisionen\n(und fünf Kavallerie-Divisionen1) den Raum nördlich des Njemen von der\nunteren Dubissa bis in die Gegend östlich von Libau in einer Frontbreite\nvon etwa 250 Kilometern. Der gegenüberstehende Feind schien an Zahl\netwas überlegen2). General von Below wollte den Angriff, ähnlich wie es\nder Oberbefehlshaber Ost seinerzeit in der Weisung vom 14. Juni3) ange-\nordnet hatte, unter Vermeidung der starken feindlichen Stellungen bei Schau-\nlen gegen den vorwiegend aus Kavallerie bestehenden russischen Nordflügel\nführen, um dann gegen Flanke und Rücken der Schaulen-Stellung einzu-\nschwenken. Dementsprechend gliederte er seine Truppen unter Schwächung\ndes rechten Flügels wie folgt:\n\nS ü d g r u p p e unter Generalleutnant Freiherr von Richthofen\n(Höherer Kavalleriekommandeur 1 

In [9]:
import json
folder_name = 'third_attempt'
with open(f'../output/{folder_name}/english_texts.json', 'w') as f:
    json.dump(english_texts, f)
with open(f'../output/{folder_name}/german_texts.json', 'w') as f:
    json.dump(german_texts, f)

save_document(german_texts, folder_name, language='German')
save_document(english_texts, folder_name, language='English')


``` 
1.  Upload Input folder of pdfs to blob storage.
2.  Read file from s3.
3.  FFT in y -> (x_hi, x_lo), write half_cropped_image to s3
4.  FFT in x -> (y_hi, y_lo), write cropped_image to s3
5.  Read cropped image from s3 -> encode_image -> translate and transcribe -> JSON output

# Define available GPT-4o models with pricing information


"gpt-4o-2024-08-06":
    "price_txt": "$2.50 / 1M input tokens"
    "price_img": "$0.001913 / 1500px^2"
    
"gpt-4o-mini-2024-07-18":
    "price_txt": "$0.150 / 1M input tokens"
    "price_img": "$0.003825 / 1500px^2"
    
```